In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
train_data = pd.read_csv(r'/kaggle/input/playground-series-s4e10/train.csv')

In [3]:
test_data = pd.read_csv(r'/kaggle/input/playground-series-s4e10/test.csv')

In [4]:
X = train_data.drop(columns = ['id','loan_status'])
y = train_data['loan_status']

In [5]:
X['income_to_laon-ratio'] =X['person_income'] / X['loan_amnt']


In [6]:
catagorical_data = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

In [7]:
for i in catagorical_data:
    X[i] = X[i].astype('category')

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.33,stratify=y, random_state=42)

In [9]:
!pip install h2o


In [10]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
X_val = h2o.H2OFrame(X_val)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp36yv2f3x
  JVM stdout: /tmp/tmp36yv2f3x/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp36yv2f3x/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_unknownUser_8imozs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
for col in catagorical_data:
    train_h2o[col] = train_h2o[col].asfactor()

In [12]:
target = 'loan_status'
features = X.columns.tolist()

In [13]:
train_h2o[target] = train_h2o[target].asfactor()

In [15]:
automl = H2OAutoML(
        max_models = 20,
        seed = 42,
        sort_metric = 'AUC',
        include_algos = ["GBM", "XGBoost", "GLM", "StackedEnsemble", "DRF"],
        balance_classes = True,
        nfolds = 5) 


In [16]:
automl.train(x=features, y=target, training_frame=train_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/20
# GBM base models (used / total),4/8
# XGBoost base models (used / total),6/9
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [27]:
test_data['income_to_laon-ratio'] =test_data['person_income'] / test_data['loan_amnt']

for col in catagorical_data:
    test_data[col] = test_data[col].astype('category')
    

In [29]:
test_h2o = h2o.H2OFrame(test_data.drop(columns=['id']))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
for col in catagorical_data:
    test_h2o[col] = test_h2o[col].asfactor()

In [33]:
best_model = automl.leader
predictions = best_model.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [34]:
output = predictions.as_data_frame()
output

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict,p0,p1
0,1,0.005914,0.994086
1,0,0.986644,0.013356
2,1,0.248353,0.751647
3,0,0.991585,0.008415
4,0,0.930359,0.069641
...,...,...,...
39093,0,0.964520,0.035480
39094,0,0.994695,0.005305
39095,0,0.984758,0.015242
39096,0,0.613167,0.386833


In [38]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e10/sample_submission.csv")
submission['loan_status']=output['p1']
submission.head()

,id,loan_status
0,58645,0.994086
1,58646,0.013356
2,58647,0.751647
3,58648,0.008415
4,58649,0.069641


In [39]:
submission.to_csv('submission.csv', index=False)